In [ ]:
import JupyterNotebooksLib as slicernb
import slicer
import vtk, ctk
from slicer.ScriptedLoadableModule import *
from slicer.util import VTKObservationMixin
from LungCTSegmenter import LungCTSegmenterLogic
import os
import glob
import ipywidgets as widgets
from ipywidgets import Layout
import zipfile as zf
import gzip
import shutil
from radiomics import featureextractor

In [ ]:
# Set viewer size to 50% of screen size
slicernb.AppWindow.setWindowSize(scale=0.5)

In [1]:
nii_file = '45.nii'

In [ ]:
slicer.mrmlScene.Clear(0)
inputVolumeNode = slicer.util.loadVolume(nii_file)

In [ ]:
# Create new markers
markupsRightLungNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
markupsRightLungNode.SetName("R")
markupsLeftLungNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
markupsLeftLungNode.SetName("L")
markupsTracheaNode = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLMarkupsFiducialNode")
markupsTracheaNode.SetName("T")

In [ ]:
# add six fiducials each right and left
markupsRightLungNode.CreateDefaultDisplayNodes()
markupsRightLungNode.AddFiducial(50.,48.,-173.)
markupsRightLungNode.AddFiducial(96.,-2.,-173.)
markupsRightLungNode.AddFiducial(92.,-47.,-173.)
markupsRightLungNode.AddFiducial(47.,-22.,-52.)
markupsRightLungNode.AddFiducial(86.,-22.,-128.)
markupsRightLungNode.AddFiducial(104.,-22.,-189.)

markupsLeftLungNode.CreateDefaultDisplayNodes()
markupsLeftLungNode.AddFiducial(-100.,29.,-173.)
markupsLeftLungNode.AddFiducial(-111.,-37.,-173.)
markupsLeftLungNode.AddFiducial(-76.,-85.,-173.)
markupsLeftLungNode.AddFiducial(-77.,22.,-55.)
markupsLeftLungNode.AddFiducial(-100.,-22.,-123.)
markupsLeftLungNode.AddFiducial(-119.,-22.,-127.)

# add one fiducial 
markupsTracheaNode.CreateDefaultDisplayNodes()
markupsTracheaNode.AddFiducial(-4.,-14.,-90.)

In [ ]:
# setup for segmenter logic
logic = LungCTSegmenterLogic()
logic.removeTemporaryObjects()
logic.rightLungFiducials = markupsRightLungNode
logic.leftLungFiducials = markupsLeftLungNode
logic.tracheaFiducials = markupsTracheaNode
logic.inputVolume = inputVolumeNode

logic.startSegmentation()
logic.updateSegmentation()
logic.applySegmentation()

In [ ]:
mask_label_path = os.path.join(cur_dir, nii_file[:-4]+'_mask-label.nrrd')
slicer.util.saveNode(logic.outputSegmentation,mask_label_path)

In [ ]:
# mask-label.nrrd file post processing
nrrd_data, nrrd_header = nrrd.read(mask_label_path)
nrrd_data[nrrd_data==2] = 1
nrrd.write(mask_label_path, nrrd_data, nrrd_header)